In [1]:
# !pip install git+https://github.com/huggingface/transformers trl accelerate torch bitsandbytes peft datasets
# !pip install pyqtwebengine
import pandas as  pd

import numpy as np
#https://medium.com/gopenai/fine-tuning-mistral-7b-instruct-model-in-colab-a-beginners-guide-0f7bebccf11c

In [2]:
import torch 
from trl import SFTTrainer
from random import randrange 
from datasets import load_dataset 
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [3]:
# Loading the dataset 
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# Since I will only finetune on Question-Answer pairs without context, I will filter accordingly
# Filter QA pairs without context
dataset = dataset.filter(lambda x:x['context'] == '')

# A prompting formatting function 
def create_prompt_instruction(sample):
   return f"""### Instruction: 
   Use the input below to create an instruction, which could have been used to generate the input using an LLM. 

   ### Input 
   {sample['response']}

   ### Response:
   {sample['instruction']}
   """

In [4]:
print(create_prompt_instruction(dataset[0]))


### Instruction: 
   Use the input below to create an instruction, which could have been used to generate the input using an LLM. 

   ### Input 
   Tope

   ### Response:
   Which is a species of fish? Tope or Rope
   


In [5]:
# # Import model and tokenizer 
# # load_in_4bit=True --> loading only 4-bit version 
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", device_map='auto',  load_in_8bit=True,use_cache=False)#load_in_4bit=True
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [6]:
modelpath="mistralai/Mistral-7B-Instruct-v0.1"

# Load 4-bit quantized model
model = AutoModelForCausalLM.from_pretrained(
    modelpath,    
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    ),
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# bitsandbytes.__version__

In [8]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [9]:
# git clone https://github.com/TimDettmers/bitsandbytes.git
# cd bitsandbytes
# CUDA_VERSION=118 make cuda11x
# python setup.py install

In [10]:
# LD_LIBRARY_PATH: export LD_LIBRARY_PATH=/usr/local/cuda-12.1/targets/x86_64-linux/lib/stubs/libcuda.so
# export LD_LIBRARY_PATH="/usr/local/cuda-12.1/targets/x86_64-linux/lib/stubs/libcuda.so:$LD_LIBRARY_PATH"
# echo $LD_LIBRARY_PATH

In [11]:
def get_prompt():
    """Select a random sample from a dataset and format it into a prompt for language model (LLM) instruction generation.

        This function randomly selects an index, retrieves a sample from a dataset, and formats a prompt string that instructs to create an LLM instruction based on the sample's response. The formatted prompt and the selected sample are returned.

        Returns:
        tuple: A tuple containing:
            - str: A formatted string that includes an instruction and the response from the randomly selected sample, intended to be used as a prompt for LLM instruction generation.
            - dict: The randomly selected sample from the dataset.

        Note:
        Ensure that 'dataset' is a pre-defined list of dictionaries, where each dictionary contains at least a key 'response' holding a text string. The function does not handle empty datasets or missing keys and may raise an exception in such cases.
        """

    idx = randrange(len(dataset))

    sample = dataset[idx]

    return f"""### Instruction: 
    Use the input below to create an instruction, which could have been used to generate the input using an LLM. 

    ### Input 
    {sample['response']}

    ### Response:
    """, sample



def get_response(prompt, sample):

    """
        Generate a response based on a given prompt using an LLM. 
    
        Parameters:
        - prompt (str): A text string to prompt LLM.
        - sample (dict): A dictionary containing a ground truth. 

        Returns:
        dict: A dictionary containing:
            - 'LLM result': The generated response from the language model, decoded from token IDs to a string.
            - 'ground truth': The ground truth instruction text extracted from the input sample.

        Note:
        Ensure that the 'tokenizer' and 'model' are loaded.
        """

    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return {
        'llm_response': decoded_output[0], # LLM-generated response 
        'reference': sample['instruction'] # Ground Truth 
    }

In [32]:
# Get the prompt first 
test_prompt, sample = get_prompt()
print(test_prompt)

# TEST PROMPT 
"""
### Instruction: 
   Use the input below to create an instruction, which could have been used to generate the input using an LLM. 

   ### Input 
   Born in Iowa, Asa Wood was a politician and newspaper publisher who later served as a state senator of Nebraska.

   ### Response:
"""

# Get the response 
response = get_response(test_prompt, sample)

# print the llm response and ground truth 
print("LLM result:")
print(response['llm_response'])
print('--'*40)
print("Ground truth:")
print(response['reference'])

### Instruction: 
    Use the input below to create an instruction, which could have been used to generate the input using an LLM. 

    ### Input 
    Patagonia branded jackets provide warmth to the wearer as well as act as fashionable styling in today's culture. The coats are lightweight and easy to pack in bags or suitcases, making them amenable for use when traveling or camping. The Patagonia corporation also donates profits heavily to charity which appeals to many consumers on the market today.

    ### Response:
    


LLM result:
<s> ### Instruction: 
    Use the input below to create an instruction, which could have been used to generate the input using an LLM. 

    ### Input 
    Patagonia branded jackets provide warmth to the wearer as well as act as fashionable styling in today's culture. The coats are lightweight and easy to pack in bags or suitcases, making them amenable for use when traveling or camping. The Patagonia corporation also donates profits heavily to charity which appeals to many consumers on the market today.

    ### Response:
    
    In recent years, Patagonia has been known for producing high-quality outdoor jackets that not only provide warmth, but also serve as fashionable attire. Made from lightweight and compact materials, these jackets are perfect for those on-the-go, whether it's for travel or camping. Additionally, Patagonia has established a strong commitment to giving back to the community by donating profits to various charitable organizations. This dedication to bo

In [ ]:
# torch.__version__

In [ ]:
# !pip install -U torch

In [33]:
ocr="""COCKTAIL
CRATE
SMALL BATCH
COCKTAIL MIX
BORN IN QUEENS NYC
CLASSIC
OLD FASHIONED
DOES NOT CONTAIN ALCOHOL,
JUST REAL INGREDIENTS LIKE
BROWN SUGAR AND BITTERS
REFRIGERATE AFTER OPENING
GMO FREE MADE WITH DEMERARA SUGAR 375 ML 12.7 FL OZ"""

In [31]:
torch.backends.cuda.enable_mem_efficient_sdp(False)

In [34]:
new_prompt="""###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim
        Condition 8 - Please do not provide any explanation, the extracted claims should be in the form of a python list for example like this "['claim 1','claim 2',....'claim n']"

### Product OCR input:
    {}
    
### Extracted Claims:
 """.format(ocr)
          
          
          
print(new_prompt)

###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim
        Condition 8 - Please do not provide any explanation, the extracted claims should be in the form of a python list for example like this "['claim 1','claim 2',....

In [35]:
test_prompt=new_prompt

In [36]:
# Get the prompt first 
# test_prompt, sample = get_prompt()
# print(test_prompt)

# TEST PROMPT
"""
### Instruction: 
   Use the input below to create an instruction, which could have been used to generate the input using an LLM. 

   ### Input 
   The Needle is a mutant supervillain created by Mark Gruenwald, Carmine Infantino, and Al Gordon. He first appeared in Spider-Woman #9 (December 1978) and was brought back during his run on the West Coast Avengers as a member of the villain team Night Shift. He was imprisoned by the Locksmith and freed by Spider-Woman. He joined the Night Shift and teamed with Captain America against the Power Broker and his augmented mutates. He also battled the West Coast Avengers, the second Hangman, and Satannish.

He was later defeated by Armory. Needle appears with the Night Shift, as part of the Hood's gang, and they battle the Midnight Sons. They are killed when the zombie virus mutates and becomes airborne. Dormammu assumes control of the Night Shift and uses them to fight the Midnight Sons. When Jennifer Kale and the Black Talon contain the virus within the Zombie, the Night Shift members are restored to normal and the Hood teleports away with them.

   ### Response:
"""

# Get the response 
response = get_response(test_prompt, sample)

# print the llm response and ground truth 
print("LLM result:")
print(response['llm_response'])
print('--'*40)
print("Ground truth:")
print(response['reference'])

LLM result:
<s> ###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim
        Condition 8 - Please do not provide any explanation, the extracted claims should be in the form of a python list for example like this "['claim 1

In [37]:
!nvidia-smi

Sat Dec 30 06:19:28 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 5000                 Off| 00000000:17:00.0 Off |                  Off |
| 34%   37C    P2               48W / 230W|   3153MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [66]:
import pandas as pd

# df=pd.read_csv("/home/niq/hcsr2001/LLM_data/claim_llm.csv")

train=pd.read_csv("/home/niq/hcsr2001/LLM_data/claim_train_llm.csv")
test=pd.read_csv("/home/niq/hcsr2001/LLM_data/claim_test_llm.csv")

In [67]:
df=train.copy()

In [68]:
text_to_add="""###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any explanation, the extracted claims should be separated by comas for example like this - claim 1,claim 2,....claim n

### Product OCR input:"""
text_to_add

'###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide any ex

In [69]:
df.head(2)

,id,Image_side,claim_0,claim_1,claim_2,claim_3,claim_4,claim_5,claim_6,claim_7,...,claim_13,claim_14,claim_15,claim_16,claim_17,claim_18,claim_19,claim_20,ocr_info,prompt_cliam
0,10305744_0,Front,REMOVE TOUGH STAINS & DIRT,NATURAL FRAGRANCE,FULL DISSOLUTION,PLANT BASED,CHILD SAFE,COLOUR SAFE,NON TOXIC,SAFE ON HANDS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIQUID\nDETERGENT\nPREMIUM\nISO\n9001:29 15\nQ...,"REMOVE TOUGH STAINS & DIRT, NATURAL FRAGRANCE..."
1,10305744_1,Back,"HARD ON DIRT AND TOUGH STAINS, MAKES YOUR CLO...",SUITABLE FOR All TYPE of CLOTHS,OKTAA active detergent LIQUID IS SUITABLE FOR...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net. Volume: 1000 ml\nM.R.P.3:\n355.00\nLDO-11...,"HARD ON DIRT AND TOUGH STAINS, MAKES YOUR CLO..."


In [70]:
import re
df["prompt_cliam"]=df["prompt_cliam"].apply(lambda x: re.sub(r',+', ',', x))

In [71]:
test["prompt_cliam"]=test["prompt_cliam"].apply(lambda x: re.sub(r',+', ',', x))

In [72]:
df["ocr_info"]=df["ocr_info"].apply(lambda x:text_to_add+"\n" +x+" \n\n ### Extracted Claims: \n")


In [73]:
test["ocr_info"]=test["ocr_info"].apply(lambda x:text_to_add+"\n" +x+" \n\n ### Extracted Claims: \n")


In [74]:
df.isnull().sum()

id                 0
Image_side         0
claim_0            0
claim_1          181
claim_2          326
claim_3          491
claim_4          636
claim_5          752
claim_6          846
claim_7          909
claim_8          959
claim_9          998
claim_10        1022
claim_11        1037
claim_12        1053
claim_13        1065
claim_14        1079
claim_15        1091
claim_16        1098
claim_17        1102
claim_18        1107
claim_19        1108
claim_20        1109
ocr_info           0
prompt_cliam       0
dtype: int64

In [75]:
from datasets import Dataset


In [76]:
df["text"]=df["ocr_info"]+" "+df["prompt_cliam"]

In [77]:
test["text"]=test["ocr_info"]+" "+test["prompt_cliam"]

In [78]:
train = Dataset.from_pandas(df[["text"]])
# test = Dataset.from_pandas(test[["text"]])

In [79]:
# dataset=dataset.train_test_split(test_size=0.02)


In [80]:
# train=dataset["train"]
train,test

(Dataset({
     features: ['text'],
     num_rows: 1110
 }),
             id Image_side                                            claim_0  \
 0    9995458_0      Front                                   LIQUID detergent   
 1    9995460_0      Front                           easy & Clear wash LIQUID   
 2    9995460_1       Back                           Gives an excellent WASH.   
 3    9999086_0      Front                                   Super Saver Pack   
 4    9999086_1       Back                                           Softouch   
 5    8389029_1       Back   our cloth wash powder is a multipurpose powde...   
 6    8981944_1       Back                                               none   
 7    8981944_6     Others                                               none   
 8    9194452_0      Front                                    4x concentrated   
 9    9218518_0      Front                                         brightness   
 10   9223912_0      Front                      

In [ ]:
# test=dataset["test"].select(np.arange(9))
# test

In [82]:
from tqdm import tqdm

In [83]:
response_list=[]
for i in tqdm(test["text"]):
    # print(" ".join(i.split("### Extracted Claims:"))+"\n\n ### Extracted Claims:")
    
    response = get_response(" ".join(i.split("### Extracted Claims:"))+"\n\n ### Extracted Claims:", sample)
    
    response_list.append(response['llm_response'])

100%|██████████| 22/22 [02:23<00:00,  6.51s/it]


In [84]:
print(" ".join(i.split("### Extracted Claims:"))+"\n\n ### Extracted Claims:")

###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any explanation, t

In [86]:
# test_complete=test.to_pandas()
test_complete=test.copy()
test_complete["prediction_Mistral"]=response_list

In [87]:
response_list

['<s> ###Instruction:\n    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims \n    Consider the below conditions before extracting the claims: \n        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation\n        Condition 2 - Consider tests and certifications mentioned as claims\n        Condition 3 - Do not consider quantity, expiry or directions to use as claims\n        Condition 4 - Do not consider any nutritional information as claims\n        Condition 5 - Do not consider contact, address or website information\n        Condition 6 - If you do not find any product claims return None\n        Condition 7 - Ignore other language claims\n        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims \n        Condition 8 - Please do not provide a

In [91]:
check_row=1

In [92]:
print(test_complete["text"].tolist()[check_row])

###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any explanation, t

In [93]:
print(test_complete["prediction_Mistral"].tolist()[check_row])

<s> ###Instruction:
    You are an expert in extracting product claims from Product OCR input of the product image, please consider the below points before extracting those claims 
    Consider the below conditions before extracting the claims: 
        Condition 1 - Extract only the English claims that the product makes and please do not provide any explanation
        Condition 2 - Consider tests and certifications mentioned as claims
        Condition 3 - Do not consider quantity, expiry or directions to use as claims
        Condition 4 - Do not consider any nutritional information as claims
        Condition 5 - Do not consider contact, address or website information
        Condition 6 - If you do not find any product claims return None
        Condition 7 - Ignore other language claims
        Condition 9 - Please do not consider volume information as claim and also please do not consider instructions provided as claims 
        Condition 8 - Please do not provide any explanatio

In [95]:
test_complete.head(2)

,id,Image_side,claim_0,claim_1,claim_2,claim_3,claim_4,claim_5,claim_6,claim_7,...,claim_15,claim_16,claim_17,claim_18,claim_19,claim_20,ocr_info,prompt_cliam,text,prediction_Mistral
0,9995458_0,Front,LIQUID detergent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,###Instruction:\n You are an expert in extr...,"LIQUID detergent,",###Instruction:\n You are an expert in extr...,<s> ###Instruction:\n You are an expert in ...
1,9995460_0,Front,easy & Clear wash LIQUID,Quick drying,removing TOUGH STAINS,Fresh SCENT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,###Instruction:\n You are an expert in extr...,"easy & Clear wash LIQUID, Quick drying, remov...",###Instruction:\n You are an expert in extr...,<s> ###Instruction:\n You are an expert in ...


In [96]:
test_complete.to_csv("/home/niq/hcsr2001/LLM_results/Mistral_LLM_res.csv",index=False)

In [ ]:
response = get_response(test_prompt, sample)

# print the llm response and ground truth 
print("LLM result:")
print(response['llm_response'])

In [ ]:
# PEFT Config 
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)


# Prepare the model for finetuning 
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
# dataset["train"],dataset["test"]

In [20]:
# Splilt dataset into 70% for training and 30% for testing 
dataset = dataset.train_test_split(test_size=0.3)

# Dataset Split 
"""
DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 7380
    })
    test: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 3164
    })
})

"""


# Mount drive to save dataset split 
# Connect colab with my drive 


# Use the train split for training 
# train_dataset = dataset['train']

"\nDatasetDict({\n    train: Dataset({\n        features: ['instruction', 'context', 'response', 'category'],\n        num_rows: 7380\n    })\n    test: Dataset({\n        features: ['instruction', 'context', 'response', 'category'],\n        num_rows: 3164\n    })\n})\n\n"

In [21]:
train_dataset = dataset['train'].select(list(np.arange(0,2000)))

In [22]:
train_dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 2000
})

In [23]:
# Define training arguments 
args = TrainingArguments(
    output_dir = "mistral_instruct_qa",
    num_train_epochs = 5,
    per_device_train_batch_size = 6,
    warmup_steps = 0.03,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    lr_scheduler_type='constant',
    disable_tqdm=True
)


# Define SFTTrainer arguments 
max_seq_length = 2048

trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
   max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=create_prompt_instruction,
    args=args,
    train_dataset=train_dataset,
)

/home/niq/hcsr2001/anaconda3/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [24]:
import torch

In [25]:
torch.__version__

'2.1.2+cu118'

In [26]:
tokenizer.pad_token = tokenizer.eos_token

In [28]:
train_dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 2000
})

In [27]:
# kick off the finetuning job 
trainer.train()

# Save finetuned model 


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/niq/hcsr2001/anaconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.00 GiB. GPU 1 has a total capacty of 15.73 GiB of which 1.27 GiB is free. Including non-PyTorch memory, this process has 14.46 GiB memory in use. Of the allocated memory 10.11 GiB is allocated by PyTorch, and 3.52 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.save_model("mistral_instruct_qa")